In [1]:
!pip install wordcloud

In [2]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import time
from nltk.corpus import wordnet
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [9]:
!pip install --upgrade nltk
!pip install --upgrade six

In [ ]:
!pip uninstall nltk
!pip install nltk

In [3]:
!pip install --upgrade spacy --user
!python -m spacy download es_core_news_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.7.0/es_core_news_sm-3.7.0-py3-none-any.whl (12.9 MB)
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [4]:
# Leemos los dataframes
tweets = pd.read_csv('tweets_extraidos_preprocesados_sentimiento.csv')

In [5]:
tweets.head()

,ID,ID_Texto,Nombre BD,Nombre Usuario,Tweet Original,Emoticones,Tweet Preprocesado,Tweet Limpio,Analisis Sentimiento,Sentimiento Score,Idioma,Retweet Count,Favorite_count,Followers_count,Friends_count,Listed_count,Favourites_count,Statuses_count
0,110825732,110825732,Cantantes_Colombianos,Camilo,Gallo pinto en Costa Rica... y otras maneras d...,NaN,gallo pinto costa rica ... manera disfrutar vi...,gallo pinto costa rica manera disfrutar vida,POS,0.998737,es,177,4650,1735142,488,1442,3246,15649
1,110825732,110825732,Cantantes_Colombianos,Camilo,"RT @rincondelatribu: La Tribu tiene casa, un h...",NaN,"rt @rincondelatribu : tribu casa , hogar , ⛺ \...",tribu casa hogar,POS,0.996282,es,44,0,1735142,488,1442,3246,15649
2,110825732,110825732,Cantantes_Colombianos,Camilo,Contando los días!! Ya quiero iiiiiiirrrrrr 🇧🇷...,NaN,contar ! ! querer iiiiiiirrrrrr 🇧 🇷 https://t....,contar querer iiiiiiirrrrrr,NEU,0.998314,es,92,830,1735142,488,1442,3246,15649
3,110825732,110825732,Cantantes_Colombianos,Camilo,En el aeropuerto en Buenos Aires por salir a c...,NaN,aeropuerto aires salir bolivia ! ! 🇧 🇴 \n diga...,aeropuerto aires salir bolivia digan reco...,NEU,0.997631,es,26,396,1735142,488,1442,3246,15649
4,110825732,110825732,Cantantes_Colombianos,Camilo,Recorriendo el mundo con mi esposa y mi hija h...,NaN,recorrer mundo esposa hija amar . cambiar . .,recorrer mundo esposa hija amar cambiar,NEU,0.595363,es,122,3286,1735142,488,1442,3246,15649


In [6]:
# Contar la cantidad de valores nulos en 'MiColumna'
cantidad_nulos_en_columna = tweets['Tweet Limpio'].isnull().sum()

# Imprimir la cantidad de valores nulos en 'MiColumna'
print("Cantidad de valores nulos en 'MiColumna':", cantidad_nulos_en_columna)


Cantidad de valores nulos en 'MiColumna': 2116


In [7]:
tweets.dropna(subset=['Tweet Limpio'], inplace=True)

In [ ]:
# Descargar el léxico VADER y las stopwords de NLTK si no están descargados previamente
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

# Crear una instancia del analizador de sentimientos VADER
sia = SentimentIntensityAnalyzer()

# Crear un lematizador de NLTK
lemmatizer = nltk.stem.WordNetLemmatizer()

# Función para realizar el análisis de sentimientos en un tweet y obtener el texto preprocesado
def analyze_sentiment_and_preprocess(tweet):
    # Tokenización con NLTK
    words = word_tokenize(tweet, language='spanish')

    # Eliminación de stopwords con NLTK
    stop_words = set(stopwords.words('spanish'))
    words = [word for word in words if word.lower() not in stop_words]

    # Lematización con NLTK
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in words if word.isalpha()]

    # Reconstrucción del texto preprocesado
    lem_tweet = ' '.join(lemmatized_tokens)

    # Aplicación del analizador de sentimientos VADER
    sentiment = sia.polarity_scores(lem_tweet)
    
    if sentiment['compound'] >= 0.05:
        sentiment_label = 'Positivo'
    elif sentiment['compound'] <= -0.05:
        sentiment_label = 'Negativo'
    else:
        sentiment_label = 'Neutral'
    
    return lem_tweet, sentiment_label

# Función para aplicar el análisis de sentimientos y obtener el texto preprocesado en paralelo
def analyze_sentiment_and_preprocess_parallel(column):
    with ThreadPoolExecutor(max_workers=4) as executor:  # Ajusta el número de hilos según tus necesidades
        results = list(tqdm(executor.map(analyze_sentiment_and_preprocess, column), total=len(column), desc="Analizando Sentimientos y Preprocesando"))
    return results

# Medir el tiempo de inicio
start_time = time.time()

# Supongamos que tienes un DataFrame llamado 'tweets' con una columna 'Tweet Limpio'
# Cambia 'tweets' y 'Tweet Limpio' por los nombres reales de tu DataFrame y columna
results = analyze_sentiment_and_preprocess_parallel(tweets['Tweet Limpio'])

# Obtener los resultados de texto preprocesado y sentimiento en listas separadas
preprocessed_text, sentiment_labels = zip(*results)

# Crear nuevas columnas en el DataFrame para el texto preprocesado y el sentimiento
tweets['Texto_Preprocesado_NLTK'] = preprocessed_text
tweets['Sentimiento_VADER_NLTK'] = sentiment_labels

# Medir el tiempo de finalización
end_time = time.time()

# Calcular el tiempo total de ejecución
total_time = end_time - start_time

# Guardar el DataFrame resultante con los textos preprocesados y el sentimiento en un nuevo archivo CSV
tweets.to_csv('tweets_con_texto_preprocesado_VADER_NLTK.csv', index=False)

# Imprimir el tiempo total de ejecución
print(f"Tiempo total de ejecución: {total_time} segundos")


In [ ]:
tweets.head()

In [ ]:
# Dividimos nuestro set de datos para obtener los datos de test
tweet_train, tweet_test = train_test_split(tweet, test_size = 0.2, random_state=RANDOM_SEED)

In [ ]:
# Etiquetas reales de sentimiento en el conjunto de datos de prueba
true_labels = ["Neutral", "Negativo", "Positivo"]

# Resultados de VADER en el conjunto de datos de prueba
vader_predictions = [analyze_vader(text) for text in tweet_test]

# Calcular métricas de evaluación
precision_vader = precision_score(true_labels, vader_predictions, average='weighted')
recall_vader = recall_score(true_labels, vader_predictions, average='weighted')
f1_vader = f1_score(true_labels, vader_predictions, average='weighted')
roc_auc_vader = roc_auc_score(true_labels, vader_predictions)

print('Resultados con VADER:')
print('Accuracy:', roc_auc_vader)
print('Precision:', precision_vader)
print('Recall:', recall_vader)
print('F1-score:', f1_vader)

In [ ]:
# Contar el número de tweets positivos
tweets_positivos = tweets[tweets['Sentimiento VADER'] == 'Positivo']

# Calcular el porcentaje de tweets positivos
porcentaje_positivos = (len(tweets_positivos) / len(tweets)) * 100

print(f"Porcentaje de tweets positivos: {porcentaje_positivos:.2f}%")

In [ ]:
# Contar el número de tweets negativos
tweets_negativos = tweets[tweets['Sentimiento VADER'] == 'Negativo']

# Calcular el porcentaje de tweets negativos
porcentaje_negativos = (len(tweets_negativos) / len(tweets)) * 100

print(f"Porcentaje de tweets negativos: {porcentaje_negativos:.2f}%")

In [ ]:
# Contar el número de tweets neutrales
tweets_neutrales = tweets[tweets['Sentimiento VADER'] == 'Neutral']

# Calcular el porcentaje de tweets negativos
porcentaje_neutrales = (len(tweets_neutrales) / len(tweets)) * 100

print(f"Porcentaje de tweets negativos: {porcentaje_neutrales:.2f}%")

In [ ]:
# Tabla que muestra la frecuencia de cada sentimiento para cada valor único de nombre_bd
tabla = pd.crosstab(tweets['Nombre BD'], tweets['Sentimiento VADER'])
print(tabla)

In [ ]:
# Gráfico de barras agrupadas para visualizar la distribución
tabla.plot(kind='bar', figsize=(12, 7))
plt.title('Distribución de Sentimientos por Base de Datos')
plt.xlabel('Nombre de la Base de Datos')
plt.ylabel('Frecuencia del Sentimeinto')
plt.legend(title='Sentimiento', loc='upper right')
plt.show()

In [ ]:
# Nube de Palabras por cada Sentimiento
# Filtra los tweets por categoría de sentimiento
tweets_positivos = tweets[tweets['Sentimiento VADER'] == 'Positivo']['Texto Preprocesado NLTK'].tolist()
tweets_negativos = tweets[tweets['Sentimiento VADER'] == 'Negativo']['Texto Preprocesado NLTK'].tolist()
tweets_neutrales = tweets[tweets['Sentimiento VADER'] == 'Neutral']['Texto Preprocesado NLTK'].tolist()

In [ ]:
# Función para crear y mostrar nubes de palabras
def generar_y_mostrar_nube_de_palabras(texto, categoria):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(texto))
    plt.figure(figsize=(8, 4))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title('Nube de Palabras - ' + categoria)
    plt.axis('off')
    plt.show()

In [ ]:
# Generar y mostrar nubes de palabras para Tweets Positivos
generar_y_mostrar_nube_de_palabras(tweets_positivos, "Positivo")

In [ ]:
# Generar y mostrar nubes de palabras para Tweets Negativos
generar_y_mostrar_nube_de_palabras(tweets_negativos, "Negativo")

In [ ]:
# Generar y mostrar nubes de palabras para Tweets Neutrales
generar_y_mostrar_nube_de_palabras(tweets_neutrales, "Neutral")